In [ ]:
from selenium import webdriver
import pickle
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import requests
from webdriver_manager.chrome import ChromeDriverManager
import pdfkit
import re

In [ ]:

class Webcrawling:
        
    def __init__(self):
        self.mfr=mfr
        self.datasheet=datasheet
        self.start_index=start_index
        self.end_index=end_index

    def splitDFIntoSmalldf(self,df):
        chunkSize = 20
        listOfDf = []
        numberChunks = len(df) // chunkSize + 1
        for i in range(numberChunks):
            listOfDf.append(df[i*chunkSize:(i+1)*chunkSize])
        return listOfDf


    def launchBrowser(self):    
        chromeOptions = webdriver.ChromeOptions()
        prefs={"profile.managed_default_content_settings.images": 2, 'disk-cache-size': 4096 }
        chromeOptions.add_experimental_option('prefs', prefs)
        driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options = chromeOptions)
        driver.get(mfr)
        return driver    
    
    
    def maxim(self):
        df = pd.read_excel(datasheet)
        df.columns = ['PTNO','LINK'] 
        df_list = self.splitDFIntoSmalldf(df[int(start_index):int(end_index)])
        print(df_list)
        driver = self.launchBrowser()
        driver.maximize_window()
        driver.implicitly_wait(5)

        under_score = ['/','?', '<', '>', '\\', ':', '*', '|', '"', ]

        for dataframe_no in range(len(df_list)):
            if (dataframe_no)%5==0:
                time.sleep(3)
                driver.get("https://www.maximintegrated.com/en/emmi/content_lookup.cfm?Ac=S")

            print('DataFrame Number: ', dataframe_no)
            url_list=[]
            for i,count in zip(df_list[dataframe_no]['PTNO'],range(len(df_list[dataframe_no]['PTNO']))):
# find Part input field
                try:
                    id_box = WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CSS_SELECTOR,"input[name='pn']")))
                    time.sleep(4)
                except:
                    time.sleep(4)
                    driver.get("https://www.maximintegrated.com/en/emmi/content_lookup.cfm?Ac=S")            
                    id_box = WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CSS_SELECTOR,"input[name='pn']")))


                id_box.clear()
                id_box.send_keys(i)
# Find submit button
                login_button = WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CSS_SELECTOR,"input[type='submit']"))).click()

                base = driver.window_handles[0]                

                try:
                    noOfRows = len(WebDriverWait(driver, 6).until(EC.presence_of_all_elements_located((By.XPATH,'/html/body/div/div[1]/div[2]/div[2]/table/tbody/tr/td/div[5]/table/tbody/tr'))))
                except:
                    noOfRows=0
                if noOfRows>0:
                    for row in range(noOfRows):
                        cell=WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.XPATH,'/html/body/div/div[1]/div[2]/div[2]/table/tbody/tr/td/div[5]/table/tbody/tr['+str(row+2)+']/td[1]/a'))).text
                        if cell == i:
                            print(cell)
                            check=1
                            WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.XPATH,'/html/body/div/div[1]/div[2]/div[2]/table/tbody/tr/td/div[5]/table/tbody/tr['+str(row+2)+']/td[1]/a'))).click()
                        break
                    if check==0:
                        print(count)
                        print('Part not found')
                        url_list.append('')
                        break

                if len(url_list)==count:
                    try:
                        WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.XPATH,'//*[@id="divContent"]/table/tbody/tr/td/table/tbody/tr[2]/td/a'))).click()
                        handles = driver.window_handles
                        driver.implicitly_wait(3)
                        driver.switch_to_window(handles[1])
                        driver.implicitly_wait(10)
                        print(count)
                        url_list.append(driver.current_url)
                        print(driver.current_url)
                        driver.implicitly_wait(2)

                        driver.close()
                        driver.switch_to_window (base)
                        
                    except:
                        print(count)
                        print("Part/PDF not found")
                        url_list.append("")
                        driver.implicitly_wait(4)

                try:
                    driver.back()
                except:
                    time.sleep(3)
                    driver.get("https://www.maximintegrated.com/en/emmi/content_lookup.cfm?Ac=S")
                    driver.implicitly_wait(10)

                while len(url_list)-count!=1:
                    url_list.pop()
            print(url_list)
            df_list[dataframe_no]['LINK'] = pd.Series(url_list).values
            print(df_list[dataframe_no])
            for i,part in zip(url_list,df_list[dataframe_no]['PTNO']):
                if len(i)!=0:
                    r = requests.get(i)     # create HTTP response object 
                                            # send a HTTP request to the server and save 
                                            # the HTTP response in a response object called r
                    for char in part:
                        if (char in under_score): 
                            part = part.replace(char,'_')
                    with open(r"PDFs_downloads\MXM\ "+str(part)+".pdf", "wb") as f:
                        f.write(r.content)


        df_mainlist = pd.concat(df_list)
        df_mainlist.to_csv('PDFs_downloads\MXM\MXM_datasheet_'+str(start_index)+'.dat',sep='|',index=False)

        driver.quit()
        
    def stmi(self):
        df = pd.read_excel(datasheet)
        df.columns = ['PTNO','LINK']

        df_list = self.splitDFIntoSmalldf(df[int(start_index):int(end_index)])

        driver = self.launchBrowser()
        driver.maximize_window()
        driver.implicitly_wait(5)
        driver.find_element_by_xpath('//*[@id="st-site"]/div[1]/div[2]/div[4]/div[2]/div').click()


        for dataframe_no in range(len(df_stmi)):
            if (dataframe_no)%5==0:
                time.sleep(3)
                driver.get("https://www.st.com/content/st_com/en.html")

            print('DataFrame Number: ', dataframe_no)
            url_list_2=[]
            for i,count in zip(df_list[dataframe_no]['PTNO'],range(len(df_list[dataframe_no]['PTNO']))):

                driver.implicitly_wait(3)
                try:
                    id_box = driver.find_element_by_css_selector("input[id='widgetSearchBar']")

                except:
                    time.sleep(4)
                    driver.get("https://www.st.com/content/st_com/en.html")    
                    driver.implicitly_wait(3)
                    id_box = driver.find_element_by_css_selector("input[id='widgetSearchBar']")


                id_box.clear()
                id_box.send_keys(i)
                
                driver.find_element_by_link_text("Search").click()
                driver.implicitly_wait(30)

                userName=driver.find_element_by_xpath('//*[@id="searchKey"]')
                driver.execute_script("arguments[0].click();", userName)
                driver.implicitly_wait(20)
                driver.find_element_by_xpath('//*[@id="search-table-products"]/tbody/tr/td[1]/a').click()
                driver.implicitly_wait(10)
                driver.find_element_by_xpath('//*[@id="st-site"]/div[6]/div[1]/main/div[1]/div[3]/div/div/div[1]/div[2]/div/div[4]/div/span').click()
                driver.implicitly_wait(30)     
                

                noOfRows = len(driver.find_elements_by_xpath('//*[@id="st-site"]/div[6]/div[1]/main/div[1]/div[4]/div[4]/div/div/div[1]/div[1]/table/tbody/tr'))
                try:
                    for row in range(noOfRows):
                        cell=driver.find_element_by_xpath('//*[@id="st-site"]/div[6]/div[1]/main/div[1]/div[4]/div[4]/div/div/div[1]/div[1]/table/tbody/tr['+str(row+1)+']/td[1]').text
                        if cell == i:
                            print(count)
                            driver.find_element_by_xpath('//*[@id="st-site"]/div[6]/div[1]/main/div[1]/div[4]/div[4]/div/div/div[1]/div[1]/table/tbody/tr['+str(row+1)+']/td[5]/div/a[1]').click()
                            driver.implicitly_wait(8)
                            url_list_2.append(driver.current_url)
                            print(driver.current_url)
                            driver.back()

                except TimeoutException:
                    print(count)
                    time.sleep(3)
                    driver.get("https://www.st.com/content/st_com/en.html")
                    url_list_2.append("")
                    driver.implicitly_wait(10)

                except:
                    print(count)
                    print('Pdf not found')
                    url_list_2.append('')

                driver.implicitly_wait(3)  

            df_stmi[dataframe_no]['LINK'] = pd.Series(url_list_2).values

            for i,part in zip(url_list_2,df_list[dataframe_no]['PTNO']):
                if len(i)!=0:
                    r = requests.get(i)
                    for char in part:
                                if (char in under_score): 
                                    part = part.replace(char,'_')
                    with open(r"PDFs_downloads\STMI\ "+str(part)+".pdf", "wb") as f:
                        f.write(r.content) 

        df_mainlist = pd.concat(df_list)
        df_mainlist.to_csv('PDFs_downloads\STMI\STMI_datasheet_'+str(start_index)+'.dat',sep='|',index=False)
        driver.quit()
        
    def cui(self):
        df = pd.read_excel(datasheet)
        df.columns = ['PART_NUMBER','LINK']
        df_list = self.splitDFIntoSmalldf(df[int(start_index):int(end_index)])
        driver = self.launchBrowser()
        driver.maximize_window()

        driver.implicitly_wait(3)

        url_list_final = []
        under_score = ['/','?', '<', '>', '\\', ':', '*', '|', '"', ]

        for dataframe_no in range(len(df_list)):
            if (dataframe_no)%5==0:
                time.sleep(3)
                driver.get("https://www.cui.com/")

            print('DataFrame Number: ', dataframe_no)
            url_list=[]
            for i,count in zip(df_list[dataframe_no]['PART_NUMBER'],range(len(df_list[dataframe_no]['PART_NUMBER']))):
                try:
                    id_box = WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[1]/div[1]/div[1]/div/div[3]/div[1]/form/input[1]')))
                except:
                    time.sleep(4)
                    driver.get("https://www.cui.com/")            
                    id_box = WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[1]/div[1]/div[1]/div/div[3]/div[1]/form/input[1]')))
                id_box.clear()
                id_box.send_keys(i)
                # Find submit button
                login_button = WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CSS_SELECTOR,"input[type='Submit']"))).click()

                base = driver.window_handles[0]   


                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"a[rel='nofollow'][href*='material-compliance']"))).click()
                    handles = driver.window_handles
                    driver.implicitly_wait(5)
                    driver.switch_to_window(handles[1])
                    driver.implicitly_wait(10)
                    print(count)
                    url_list.append(driver.current_url)
                    print(driver.current_url)
                    driver.implicitly_wait(2)

                    driver.close()
                    driver.switch_to_window (base)

                except:
                    print(count)
                    print("Part/PDF not found")
                    url_list.append('')   
                  
                try:
                    driver.back()

                except TimeoutException:
                    time.sleep(3)
                    driver.get("https://www.cui.com/")
                    driver.implicitly_wait(10)

            df_list[dataframe_no]['LINK'] = pd.Series(url_list).values
            for i,part in zip(url_list, df_list[dataframe_no]['PART_NUMBER']):
                if i:
                    pdfkit.from_url(i, "PDFs_downloads\CUI\ "+str(part)+".pdf")

            url_list_final.append(url_list)

        driver.quit()
        df_mainlist = pd.concat(df_list)
        df_mainlist.to_csv('PDFs_downloads\CUI\CUI_datasheet_'+str(start_index)+'.dat',sep='|',index=False)

In [ ]:
def main():
    mfr=input('Enter Manufacturer Website: ')
    datasheet=input('Enter Datasheet Path: ')
    start_index = input('Enter Starting Row: ')
    end_index = input('Enter Ending Row: ')

    webcrawl = Webcrawling()
    if mfr=="https://www.maximintegrated.com/en/emmi/content_lookup.cfm?Ac=S":
        webcrawl.maxim()
    if mfr=="https://www.st.com/content/st_com/en.html":
        webcrawl.stmi()
    if mfr=="https://www.cui.com/":    
        webcrawl.cui()

if __name__ == '__main__':
    main()

Enter Manufacturer Website: https://www.cui.com/
Enter Datasheet Path: C:\Users\arushi.sharma\Documents\WebCrawling\CUI.xlsx
Enter Starting Row: 4
Enter Ending Row: 6

Checking for win32 chromedriver:75.0.3770.140 in cache
Driver found in C:\Users\arushi.sharma\.wdm\chromedriver\75.0.3770.140\win32/chromedriver.exe


c:\users\arushi.sharma\appdata\local\programs\python37\myenv\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: use options instead of chrome_options


DataFrame Number:  0


c:\users\arushi.sharma\appdata\local\programs\python37\myenv\lib\site-packages\ipykernel_launcher.py:273: DeprecationWarning: use driver.switch_to.window instead


0
https://www.cui.com/product/resource/material-compliance/mj1-2502


c:\users\arushi.sharma\appdata\local\programs\python37\myenv\lib\site-packages\ipykernel_launcher.py:281: DeprecationWarning: use driver.switch_to.window instead


1
Part/PDF not found
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
